In [20]:
# Import the python library that connects to CoppeliaSim, the file is sim.py.
import sys

try:
    import sim
except Exception as e:
    print(e)
    print('--------------------------------------------------------------')
    print('"sim.py" could not be imported. This means very probably that')
    print('either "sim.py" or the remoteApi library could not be found.')
    print('Make sure both are in the same folder as this file,')
    print('or appropriately adjust the file "sim.py"')
    print('--------------------------------------------------------------')
    print('')

import time

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [21]:
class robot():
    
    def __init__(self, motor_names, frame_name=None):        
        self.client_id = self.open_connection()
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  None
        if frame_name is not None:
            self.frame = self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        clientID = self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object):
        relative_handle = self._get_handler(relative_object)
        sim.simxSetObjectPosition(clientID, self.frame, relative_handle, position, sim.simx_opmode_oneshot)
    

        

In [44]:
def point_to_point_traj(x1, x2, v1, v2, delta_t):
  t = np.linspace(0, delta_t, 100)  
  a0 = x1
  a1 = v1
  a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
  a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)

  polynomial = a0 + a1 * t + a2 * t**2 + a3 * t**3
  derivative = a1 + 2*a2 * t + 3 * a3 * t**2
  return polynomial, derivative


def piecewise2D (X,Y, Vx, Vy, T):
    theta_x, theta_y, dx, dy = [], [], [], []

    for i in range(len(P)-1):          
        theta_xi, dxi = point_to_point_traj(X[i], X[i+1], Vx[i], Vx[i+1], T[i+1] - T[i])
        theta_yi, dyi = point_to_point_traj(Y[i], Y[i+1], Vy[i], Vy[i+1], T[i+1] - T[i])
        theta_x += theta_xi.tolist()
        theta_y += theta_yi.tolist()
        dx += dxi.tolist()
        dy += dyi.tolist()
#         print("Theta_X:", theta_xi.tolist())
#         print("Theta_Y:", theta_yi.tolist())
        print("dx: ", dxi.tolist())
        print("dy: ", dyi.tolist())

        plot(theta_xi, theta_yi)
    return theta_x, theta_y, dx, dy

In [67]:
##### Requirements for the trajectory

# Waypoints
p1 = [7.6250,8.5500]
p2 = [.4000, 8.0500]
p3 = [-4, 4]
p3 = [-9,6]
p4 = [3,6]
p5 = [3,0]
p6 = [-1,0]
p7 = [-1,10]
pf = [9,10]


r1 = 0
r2 = 90
# Velocities
#X Y
v1 = [1, 0]
v2 = [0, -1]
v3 = [1.5,-1]
v4 = [2,0]
v5 = [1,1.35]
v6 = [0, 10]
v7 = [5,0]
vf = [0,-0]


# Time
t1 = 1
t2 = 20
t3 = 35
t4 = 37
t5 = 50
t6 = 52
t7 = 62
t8 = 72

V = np.vstack((v1, v2))
T = [t1, t2]
P = np.vstack((p1, p2))
R = [r1, r2]

X, Y = P[:,0], P[:,1]
Vx, Vy = V[:,0], V[:,1]



# theta_x, theta_y, dx, dy = piecewise2D (X,Y, Vx, Vy, T)
# for v_x, v_y in zip(dx, dy):
#     if r.client_id == -1:  # Check connection
#         print('Robot is not connected')
#         break
#     vx, vy = v_y, -v_x
#     print(vx, ", ", vy)
#     r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
#     time.sleep(.1)
        
r.send_motor_velocities([0, 0, 0, 0])


In [68]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot(motor_names)  # Create an instance of our robot

prev_t = 0;
for (t, v_x, v_y, r) in zip(T, Vx, Vy, R):
    if r.client_id == -1:  # Check connection
        print('Robot is not connected')
        break
    
    if(r not 0):
        r.send_motor_velocities([r, 0,0,0])
    
    vx, vy = v_x, v_y
    print(vx, vy)
    r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    time.sleep(t-prev_t)
    prev_t = t


trajectories = []

    
r.send_motor_velocities([0, 0, 0, 0])

r.close_connection()  # End

SyntaxError: invalid syntax (<ipython-input-68-a8ddb11b3556>, line 11)